In [41]:
import pandas as pd
import numpy as np

In [42]:
dataset=pd.read_csv("spacenews.csv")

In [43]:
dataset.head()

,title,url,content,author,date,postexcerpt
0,Electron launches first NASA PREFIRE Earth sci...,https://spacenews.com/electron-launches-first-...,LOS ANGELES — Rocket Lab launched the first of...,Jeff Foust,"May 25, 2024",NaN
1,Space Development Agency to procure dedicated ...,https://spacenews.com/space-development-agency...,WASHINGTON — T he Space Development Agency (S...,Sandra Erwin,"May 25, 2024",NaN
2,Bringing generative artificial intelligence to...,https://spacenews.com/bringing-generative-arti...,"TAMPA, Fla. — Amazon Web Services is busy posi...",Jason Rainbow,"May 24, 2024",NaN
3,SpaceX sets early June launch of next Starship...,https://spacenews.com/spacex-sets-early-june-l...,LOS ANGELES — SpaceX has set a June 5 launch d...,Jeff Foust,"May 24, 2024",NaN
4,NASA and Boeing moving ahead with Starliner te...,https://spacenews.com/nasa-and-boeing-moving-a...,LOS ANGELES — NASA and Boeing are preparing fo...,Jeff Foust,"May 24, 2024",NaN


In [44]:
dataset.columns

Index(['title', 'url', 'content', 'author', 'date', 'postexcerpt'], dtype='object')

In [45]:
news_dataset=dataset[["title","author","content","date"]]
news_dataset.head()

,title,author,content,date
0,Electron launches first NASA PREFIRE Earth sci...,Jeff Foust,LOS ANGELES — Rocket Lab launched the first of...,"May 25, 2024"
1,Space Development Agency to procure dedicated ...,Sandra Erwin,WASHINGTON — T he Space Development Agency (S...,"May 25, 2024"
2,Bringing generative artificial intelligence to...,Jason Rainbow,"TAMPA, Fla. — Amazon Web Services is busy posi...","May 24, 2024"
3,SpaceX sets early June launch of next Starship...,Jeff Foust,LOS ANGELES — SpaceX has set a June 5 launch d...,"May 24, 2024"
4,NASA and Boeing moving ahead with Starliner te...,Jeff Foust,LOS ANGELES — NASA and Boeing are preparing fo...,"May 24, 2024"


In [46]:
news_dataset.isnull().sum()

title        0
author       0
content    165
date         0
dtype: int64

In [47]:
news_dataset.dropna(inplace=True)

C:\Users\miana\AppData\Local\Temp\ipykernel_13572\478739985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset.dropna(inplace=True)


In [48]:
news_dataset.isnull().sum()

title      0
author     0
content    0
date       0
dtype: int64

In [49]:
import re
def clean_text(text):
    text=text.lower()
    text=re.sub(r'[^a-zA-z0-9\s]',' ',text)
    text=re.sub(r'\s+',' ',text)
    return text

In [50]:
news_dataset["clean_content"]=news_dataset["content"].apply(clean_text)

C:\Users\miana\AppData\Local\Temp\ipykernel_13572\3916579409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset["clean_content"]=news_dataset["content"].apply(clean_text)


In [51]:
news_dataset.head()

,title,author,content,date,clean_content
0,Electron launches first NASA PREFIRE Earth sci...,Jeff Foust,LOS ANGELES — Rocket Lab launched the first of...,"May 25, 2024",los angeles rocket lab launched the first of t...
1,Space Development Agency to procure dedicated ...,Sandra Erwin,WASHINGTON — T he Space Development Agency (S...,"May 25, 2024",washington t he space development agency sda a...
2,Bringing generative artificial intelligence to...,Jason Rainbow,"TAMPA, Fla. — Amazon Web Services is busy posi...","May 24, 2024",tampa fla amazon web services is busy position...
3,SpaceX sets early June launch of next Starship...,Jeff Foust,LOS ANGELES — SpaceX has set a June 5 launch d...,"May 24, 2024",los angeles spacex has set a june 5 launch dat...
4,NASA and Boeing moving ahead with Starliner te...,Jeff Foust,LOS ANGELES — NASA and Boeing are preparing fo...,"May 24, 2024",los angeles nasa and boeing are preparing for ...


In [52]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# embeddings=model.encode(news_dataset["clean_content"].values)

In [ ]:
# embeddings=np.array(embeddings)
# np.save("content_embeddings.npy",embeddings)

In [53]:
embeddings=np.load("content_embeddings.npy")

In [ ]:
import faiss
dimension=embeddings.shape[1]
faiss_index=faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)
# faiss.write_index(faiss_index,"content_faiss_index.index")

In [ ]:
def get_similar_news(query,model=model,faiss_index=faiss_index,count=5):
    query_embedding=model.encode([query])
    distance,indices=faiss_index.search(query_embedding,count)
    results=[]
    for i in range(count):
        results.append(news_dataset["content"].iloc[indices[0][i]])
        return results

In [86]:
get_similar_news("SpaceX launches new satellite into orbit")


News1: WASHINGTON — SpaceX has completed the first reusable orbital launch since the retirement of the U.S. space shuttle, delivering the SES-10 telecommunications satellite into geostationary transfer orbit with a rocket that first flew last April for NASA. SpaceX launched the Falcon 9 at 6:27 p.m. Eastern from Launch Complex 39A at Kennedy Space Center, achieving a historic milestone for the company that hung its cape on the claim that rockets can be re-flown, and that reusability is necessary to take humanity to Mars.  SES entrusted SpaceX with SES-10, a 5,300-kilogram satellite equipped with 55 Ku-band beams for television broadcast and internet connectivity across Latin America. SES-10 is the second satellite the Luxembourg-based satellite operator flew on what would classify as an inaugural flight for SpaceX. In 2013, SpaceX launched SES-8 on a Falcon 9, marking the first time the rocket ever delivered a spacecraft to geostationary transfer orbit. All previous Falcon 9 missions h